# RT - class tutorial
Author: Robert Szmurło

The RT class is designed to facilitiate the operations on RT (Radiotherapy data). The class assumes that the data is organized in teh following structure at some folder:

```
./input  - folder with all input data
./input/dicom - folder with dicom files (the CT files, RS - ROI structure file, RP - plan file and RD - dose files are required)
./processing
./output
```

The class collects infomration about th eplanning grid from the RD (dose file).

In the following tutorial we'll learn typical operations that can be run using the class.

Assuming that you are starting the notebook in the `bdcalc/notebooks` folder. The first thing, we need to add PYTHONPATH to src folder and import the main RT class.

In [49]:
import sys
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact
sys.path.append("../src")
import bdcalc2.bdcalc2
#from bdcalc2.bdcalc2 import RT

In [58]:
import importlib
importlib.reload(bdcalc2.bdcalc2)

<module 'bdcalc2.bdcalc2' from '../src/bdcalc2/bdcalc2.py'>

## Initialization

To initialize the class you need to pass the folder with the data (the ./input/dicom/... is the most important)

In [59]:
rt = bdcalc2.bdcalc2.RT("/doses-nfs/sim/HN_1")

## CT data visualization

First we shall visualize some CT data. We will read the CT files and convert the images into three dimensional cube of voxels. Let we read the CT data and investigate its dimensions (these are the index coordinates).

Please note, that the first dimension in the shape refers to the number of images - the Z coordinate, second to Y and last one to X.

In [13]:
act = rt.get_ct()
act.shape

(400, 512, 512)

To get information about geometrical position and grid spacing we may issue the command:

In [52]:
rt.get_ct_griddata()

(-30.0, -30.0, -21.25, 0.1171875, 0.1171875, 0.125, 512, 512, 400)

We can get the CT grid information as:

In [53]:
print(f"Origin of the gCT grid: {rt.cto()}") 
print(f"Spacing of the CT grid: {rt.ctdx(),rt.ctdy(),rt.ctdz()}") 
print(f"Dimensions of the CT grid (nx,ny,nz): {rt.ctn()}") 
print(f"Dimensions of the CT grid (nz,ny,nx): {rt.ctn(inverted=True)}") 

Origin of the gCT grid: (-30.0, -30.0, -21.25)
Spacing of the CT grid: (0.1171875, 0.1171875, 0.125)
Dimensions of the CT grid (nx,ny,nz): (512, 512, 400)
Dimensions of the CT grid (nz,ny,nx): (400, 512, 512)


In [60]:
planct = rt.get_ct(plan=True)
planct.shape

(196, 134, 231)

Now we are ready to visualiza the CT sampled over planning grid.

In [61]:
@interact(z=(0,planct.shape[0]-1))
def f(z):
    fig,ax = plt.subplots(figsize=(16,9), dpi=50)
    ax.imshow(planct[z,:,:])
    fig.show()


interactive(children=(IntSlider(value=97, description='z', max=195), Output()), _dom_classes=('widget-interact…

## Working with ROIs

Now we are ready to view the ROI markers mapped on the voxels grid.

First, let we show the ROIs identified in the DICOM files and with assigned BIT number.

In [63]:
rt.get_rtss()

{28: {'name': 'Brainstem+3mm', 'roi_bit': 1},
 23: {'name': 'Mandible', 'roi_bit': 2},
 20: {'name': 'Spinal Cord+3mm', 'roi_bit': 4},
 19: {'name': 'Eye Globe R', 'roi_bit': 8},
 18: {'name': 'Eye Globe L', 'roi_bit': 16},
 17: {'name': 'ptv54', 'roi_bit': 32},
 16: {'name': 'ctv54', 'roi_bit': 64},
 15: {'name': 'Parotid Gland R', 'roi_bit': 128},
 14: {'name': 'Parotid Gland L', 'roi_bit': 256},
 12: {'name': 'Spinal Cord', 'roi_bit': 512},
 11: {'name': 'ptv66', 'roi_bit': 1024},
 10: {'name': 'ptv60', 'roi_bit': 2048},
 8: {'name': 'Brainstem', 'roi_bit': 4096},
 5: {'name': 'gtv6600', 'roi_bit': 8192},
 3: {'name': 'ctv60', 'roi_bit': 16384},
 1: {'name': 'Patient Outline', 'roi_bit': 32768},
 29: {'name': 'NT', 'roi_bit': 65536},
 34: {'name': 'pterygoid P', 'roi_bit': 131072},
 33: {'name': 'pterygoid L', 'roi_bit': 262144},
 32: {'name': 'Esophagus', 'roi_bit': 524288},
 31: {'name': 'Larynx', 'roi_bit': 1048576},
 30: {'name': 'Oral Cavity', 'roi_bit': 2097152}}

In [65]:
np.prod(rt.ctn())*8

838860800

## Displaying ROI mapped over CT grid

Warning! The first call of this function may take significant time as the voxel labeling is time consuming. The next time the method will use the cached data (be careful! the cache data takes a lot of disk space: `np.prod(rt.ctn())*8` space!).

In [69]:
print(f"Approximate size of cache file in {np.prod(rt.ctn())*8 / 1e6} MB")

Approximate size of cache file in 838.8608 MB


In [70]:
# this may take time to calculate for the first before the cached data is created in the ./processing folder
rois_on_ct = rt.get_roi_labels_ct()
rois_on_ct.shape

(400, 512, 512)

In [150]:
display()


['Brainstem+3mm',
 'Mandible',
 'Spinal Cord+3mm',
 'Eye Globe R',
 'Eye Globe L',
 'ptv54',
 'ctv54',
 'Parotid Gland R',
 'Parotid Gland L',
 'Spinal Cord',
 'ptv66',
 'ptv60',
 'Brainstem',
 'gtv6600',
 'ctv60',
 'Patient Outline',
 'NT',
 'pterygoid P',
 'pterygoid L',
 'Esophagus',
 'Larynx',
 'Oral Cavity']

In [155]:
from matplotlib import cm

ct = rt.get_ct()
rois_on_ct = rt.get_roi_labels_ct()

palette = cm.get_cmap('tab20')


@interact(z=(0,ct.shape[0]-1), rn=[n[1]['name'] for n in rt.get_rtss().items()])
def f(z=ct.shape[0]//2, rn='Patient Outline'):
    roi_bit = next(filter(lambda v: v[1]["name"] ==rn, rt.get_rtss().items()))[1]["roi_bit"]
    fig,ax = plt.subplots(figsize=(16,9), dpi=100)
    ax.imshow(ct[z,:,:], cmap='bone')
    rm =  np.ma.masked_where(np.bitwise_and(rois_on_ct, roi_bit) != roi_bit, 2*np.ones(rois_on_ct.shape))[z,:,:]
    #rm = (np.ma.masked_where( np.bitwise_and(rois_on_ct, 32768) == 32768, rois_on_ct / 32768))[z,:,:]
    ax.imshow(rm,interpolation='nearest',
              cmap=palette,
              norm=colors.Normalize(vmin=0.0, vmax=20.0),
              alpha=0.5)
    fig.show()


interactive(children=(IntSlider(value=200, description='z', max=399), Dropdown(description='rn', index=15, opt…

## Display rois on PLAN grid

In [159]:
from matplotlib import cm

planct = rt.get_ct(plan=True)
rois_on_plan = rt.get_roi_labels_plan()

palette = cm.get_cmap('tab20')

@interact(z=(0,planct.shape[0]-1), rn=[n[1]['name'] for n in rt.get_rtss().items()])
def f(z=planct.shape[0]//2, rn='Patient Outline'):
    roi_bit = next(filter(lambda v: v[1]["name"] ==rn, rt.get_rtss().items()))[1]["roi_bit"]
    fig,ax = plt.subplots(figsize=(16,9), dpi=100)
    ax.imshow(planct[z,:,:], cmap='bone')
    rm =  np.ma.masked_where(np.bitwise_and(rois_on_plan, roi_bit) != roi_bit, 2*np.ones(rois_on_plan.shape))[z,:,:]
    ax.imshow(rm,interpolation='nearest',
              cmap=palette,
              norm=colors.Normalize(vmin=0.0, vmax=20.0),
              alpha=0.5)
    fig.show()


interactive(children=(IntSlider(value=98, description='z', max=195), Dropdown(description='rn', index=15, opti…

In [48]:
dir(rt)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_ctdata_array',
 '_ctdata_plan_array',
 '_ctgriddata',
 '_find_dicoms',
 '_plangriddata',
 '_rdata_ct',
 '_rdata_plan',
 '_rtss',
 'ct_x_range',
 'ct_y_range',
 'ct_z_range',
 'ctd',
 'ctdx',
 'ctdy',
 'ctdz',
 'ctn',
 'ctnx',
 'ctny',
 'ctnz',
 'cto',
 'ctox',
 'ctoy',
 'ctoz',
 'dicom_files',
 'get_ct',
 'get_ct_griddata',
 'get_plan_grid_points',
 'get_plan_griddata',
 'get_roi_labels_ct',
 'get_roi_labels_plan',
 'get_rtss',
 'map_ctsopid',
 'plan_z_range',
 'pland',
 'plandx',
 'plandy',
 'plandz',
 'plann',
 'plannx',
 'planny',
 'plannz',
 'plano',
 'planox',
 'planoy',
 'planoz',
 'point2ct_idx',
 'point2plan_idx